In [191]:
import pandas as pd

# -- Selenium --
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from time import sleep
from selenium.webdriver.common.by import By


# -- API --
import re
import json
import requests

# Get today
from datetime import date

# Check time
from tqdm import tqdm

In [192]:
# driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/hệ khuyến nghị/đồ án/crawl_data_foody/chromedriver")

In [193]:
# Auto Login or Pass the block from Shopee

def login_account(driver):
    link_login = r'https://shopee.vn/buyer/login?next=https%3A%2F%2Fshopee.vn%2F'
    driver.get(link_login)

    if(len(driver.find_elements(by=By.NAME, value='loginKey'))):
        _username = ''
        _password = ''

        username_field = driver.find_element(by=By.NAME, value='loginKey')
        username_field.send_keys(_username)

        password_field = driver.find_element(by=By.NAME, value='password')
        password_field.send_keys(_password)

        sleep(0.5)

        login_button = driver.find_element(by=By.XPATH, value='//button[@class="wyhvVD _1EApiB hq6WM5 L-VL8Q cepDQ1 _7w24N1"]')
        login_button.click()

        sleep(10)

def check_login_account(driver):
    if(len(driver.find_elements(by=By.NAME, value='loginKey'))):
        _username = ''
        _password = ''

        username_field = driver.find_element(by=By.NAME, value='loginKey')
        username_field.send_keys(_username)

        password_field = driver.find_element(by=By.NAME, value='password')
        password_field.send_keys(_password)

        sleep(0.5)

        login_button = driver.find_element(by=By.XPATH, value='//button[@class="wyhvVD _1EApiB hq6WM5 L-VL8Q cepDQ1 _7w24N1"]')
        login_button.click()

        sleep(10)

def get_cmt_counts(driver):
    try:
        check_cmt_counts = driver.find_element(by=By.XPATH, value="//div[@class='product-rating-overview__filter product-rating-overview__filter--with-comment']").text
    except:
        # product-rating-overview__filter product-rating-overview__filter--active product-rating-overview__filter--with-comment
        check_cmt_counts = driver.find_element(by=By.XPATH, value="//div[@class='product-rating-overview__filter product-rating-overview__filter--active product-rating-overview__filter--with-comment']").text
    cmt_counts = 0
    if('k' in check_cmt_counts):
        get_number_in_check_cmt_counts = check_cmt_counts.split('Có Bình Luận (')[1].replace(')','').replace(',','.')
        cmt_counts = int(float(get_number_in_check_cmt_counts.replace('k','')) * 1000)
    elif('k' not in check_cmt_counts):
        get_number_in_check_cmt_counts = check_cmt_counts.split('Có Bình Luận (')[1].replace(')','')
        cmt_counts = int(get_number_in_check_cmt_counts)
    return cmt_counts

check link crawl

In [194]:
path_data = 'C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetLink_step_1/La Roche-Posay.csv'
data_link = pd.read_csv(path_data)
data_link = data_link[100:].link

In [195]:
data_link

100    https://shopee.vn/B%E1%BB%99-s%E1%BA%A3n-ph%E1...
Name: link, dtype: object

In [196]:
def startCrawl(data_link, TotalCmtInfo):
    driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/hệ khuyến nghị/đồ án/crawl_data_foody/chromedriver")
    sleep(3)
    login_account(driver)

    for link in data_link:
        driver.get(link)

        sleep(8)


        driver.execute_script("window.scrollTo(0, 2400);")
        sleep(1)

        # click to button "Have Comment"
        haveCmtButton = driver.find_element(by=By.XPATH, value = '//div[@class="product-rating-overview__filter product-rating-overview__filter--with-comment"]')
        haveCmtButton.click()
        sleep(1)

        # Define button continue
        try:
            driver.execute_script("window.scrollTo(0, 2500);")
            buttonContinue = driver.find_element(by=By.XPATH, value='//button[@class="shopee-icon-button shopee-icon-button--right "]')
        except:
            driver.execute_script("window.scrollTo(0, 2700);")
            buttonContinue = driver.find_element(by=By.XPATH, value='//button[@class="shopee-icon-button shopee-icon-button--right "]')
        # Check total cmt
        num_cmt = get_cmt_counts(driver)

        # If total cmt < total crawled cmt => stop crawl
        while(len(TotalCmtInfo) < (num_cmt - 1)):
            numReviewNow = len(driver.find_elements(by=By.XPATH, value='//div[@class="shopee-product-rating__rating"]'))
            

            for i in range(numReviewNow):

                
                CmtInfo = {}
                CmtInfo['reviewStar'] = len(driver.find_elements(by=By.XPATH, value='//div[@class="shopee-product-rating__rating"]')[i].find_elements(by=By.CSS_SELECTOR, value = '.icon-rating-solid'))
                
                try:
                    CmtInfo['comment'] = driver.find_elements(by=By.XPATH, value='//div[@class="EXI9SU"]')[i].text
                except:
                    CmtInfo['comment'] = ''
                
                CmtInfo['timeCmt'] = driver.find_elements(by=By.XPATH, value='//div[@class="shopee-product-rating__time"]')[i].text
                CmtInfo['nameProduct'] = driver.find_element(by=By.XPATH, value="//div[@class='YPqix5']/span").text
                CmtInfo['linkProduct'] = link


                TotalCmtInfo.append(CmtInfo)
                
            print(len(TotalCmtInfo))
            
            buttonContinue.click()
            sleep(2.5)
        



Chinh so luong crawl

In [197]:
import threading
import numpy as np

list_threads = []
TotalCmtInfo = []
num_thread = 2
list_partitions = [partition for partition in np.array_split(data_link, num_thread)]
for partition_list in list_partitions:
    list_threads.append(threading.Thread(target=startCrawl, args=(partition_list, TotalCmtInfo)))

In [198]:
for thread in list_threads:
    thread.start()  
    time.sleep(20)  

C:\Users\Admin\AppData\Local\Temp\ipykernel_17044\2911266436.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/hệ khuyến nghị/đồ án/crawl_data_foody/chromedriver")
Exception in thread Thread-26 (startCrawl):
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_17044\2911266436.py", line 4, in startCrawl
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_17044\2192570220.py", line 5, in login_account
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 437, in get
    self.execute

In [199]:
for thread in list_threads:
    thread.join()  

In [200]:
len(TotalCmtInfo)

0

Chinh ten file dau ra

In [201]:
pd.DataFrame(TotalCmtInfo).to_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_100_100.csv', index = False)

In [ ]:
df_check = pd.DataFrame(TotalCmtInfo)

df_check[df_check['reviewStar'] == 1]

In [ ]:
df_check['reviewStar'].value_counts()

5    142
4      2
1      2
Name: reviewStar, dtype: int64

In [ ]:
df_check['linkProduct'].value_counts()

https://shopee.vn/B%E1%BB%99-s%E1%BA%A3n-ph%E1%BA%A9m-x%E1%BB%8Bt-kho%C3%A1ng-gi%C3%BAp-l%C3%A0m-s%E1%BA%A1ch-l%C3%A0m-d%E1%BB%8Bu-da-La-RochePosay-Serozinc-i.37251700.13980697858?sp_atk=f085584a-4a75-411e-b84c-08f8d355c959    146
Name: linkProduct, dtype: int64

In [ ]:
# driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/hệ khuyến nghị/đồ án/crawl_data_foody/chromedriver")

In [ ]:
# driver.get('https://shopee.vn/S%E1%BB%AFa-r%E1%BB%ADa-m%E1%BA%B7t-t%E1%BA%A9y-trang-d%E1%BB%8Bu-nh%E1%BA%B9-cho-da-r%E1%BA%A5t-nh%E1%BA%A1y-c%E1%BA%A3m-v%C3%A0-k%C3%ADch-%E1%BB%A9ng-La-Roche-Posay-Toleriane-Dermo-Hydrating-100ml-i.37251700.14103877301?sp_atk=2ad6c8d2-eb2f-4cc2-8495-c652bf92efc4&xptdk=2ad6c8d2-eb2f-4cc2-8495-c652bf92efc4')

In [4]:
import pandas as pd

df_1 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_0_3.csv')
df_2 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_4_7.csv')
df_3 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_8_11.csv')
df_4 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_11_13.csv')
df_5 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_14_16.csv')
df_6 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_17_19.csv')
df_7 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_19_21.csv')
df_8 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_22_24.csv')
df_9 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_25_27.csv')
df_10 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_28_30.csv')
df_11 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_31_33.csv')
df_12 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_34_36.csv')
df_13 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_37_38.csv')
df_14 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_39_39.csv')
df_15 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_40_42.csv')
df_16 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_43_45.csv')
df_17 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_45_45.csv')
df_18 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_46_48.csv')
df_19 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_49_51.csv')
df_20 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_51_52.csv')
df_21 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_53_54.csv')
df_22 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_55_56.csv')
df_23 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_57_58.csv')
df_24 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_59_60.csv')
df_25 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_61_62.csv')
df_26 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_63_64.csv')
df_27 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_65_66.csv')
df_28 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_67_67.csv')
df_29 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_68_68.csv')
df_30 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_69_69.csv')
df_31 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_70_70.csv')
df_32 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_71_73.csv')
df_33 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_74_75.csv')
df_34 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_76_77.csv')
df_35 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_78_79.csv')
df_36 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_80_81.csv')
df_37 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_82_83.csv')
df_38 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_84_84.csv')
df_39 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_85_85.csv')
df_40 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_86_87.csv')
df_41 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_88_89.csv')
df_42 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_90_91.csv')
df_43 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_92_93.csv')
df_44 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_94_95.csv')
df_45 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_96_97.csv')
df_46 = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetCmt_step_2/cmtShopeeStep2_98_99.csv')




In [ ]:
pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,
            df_11,df_12,df_13,df_14,df_15,df_16,df_17,df_18,df_19,df_20,
            df_21,df_22,df_23,df_24,df_25,df_26,df_27,df_28,df_29,df_30,
            df_31,df_32,df_33,df_34,df_35,df_36,df_37,df_38,df_39,df_40,])